In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [12]:
d = 10
r = 5 
n = 1000
def Coordinate_descend_genep(d, r):
    ide = np.eye(d)
    col_num = np.arange(d)
    select_col = np.random.choice(col_num, r, replace=False)
    sign = np.random.choice([-1, 1], r)
    P = np.sqrt(d / r) * ide[:, select_col] * sign
    return P
sum_ptp = np.zeros((r, r))
sum_ppt = np.zeros((d, d))
sum_p = np.zeros((d, r))
for i in range(n):
    p = Coordinate_descend_genep(d, r)
    sum_p += p
    sum_ptp += p.T @ p
    sum_ppt += p @ p.T
sum_ptp /= n
sum_ppt /= n
sum_p /= n

In [14]:
d = 5
r = 3
n = 1000
def Spherical_smoothing_genep(d, r):
    z = np.random.randn(d, d)
    Q, R = np.linalg.qr(z)
    D = np.diag(np.sign(np.diag(R)))
    Q = Q @ D
    R = D @ R
    assert np.allclose(Q @ R, z, atol=1e-7), "the QR decomposion is not accuracy"
    P = np.sqrt(d / r) * Q[:, :r]
    return P
sum_ptp = np.zeros((r, r))
sum_ppt = np.zeros((d, d))
for i in range(n):
    p = Spherical_smoothing_genep(d, r)
    sum_ptp += p.T @ p
    sum_ppt += p @ p.T
sum_ptp /= n
sum_ppt /= n


In [16]:
sum_ppt

array([[ 1.02078913,  0.00281439,  0.00390279,  0.00926093, -0.02146264],
       [ 0.00281439,  1.01920871, -0.00904708,  0.01339887, -0.01596834],
       [ 0.00390279, -0.00904708,  0.97774856, -0.00870662,  0.01479128],
       [ 0.00926093,  0.01339887, -0.00870662,  0.98127913,  0.00355863],
       [-0.02146264, -0.01596834,  0.01479128,  0.00355863,  1.00097447]])

In [6]:
from transformers import (
    LlamaConfig,
    LlamaForCausalLM,
    AutoTokenizer,
)
import torch
import torch.nn as nn

In [3]:
config = LlamaConfig.from_pretrained("./configs/llama_60m.json")
model = LlamaForCausalLM(config)
optim = torch.optim.SGD(model.parameters())


/home/xuyuqi/.conda/envs/minimind/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:606: UserWarning: `pad_token_id` should be positive but got -1. This will cause errors when batch generating, if there is padding. Please set `pad_token_id` explicitly as `model.generation_config.pad_token_id=PAD_TOKEN_ID` to avoid errors in generation
  warnings.warn(


In [1]:
#def print_name(model):
    #for name, child in model.named_children():
        #if isinstance(child, nn.Linear):
            #print(name)
        #else:
            #print_name(child)
#print_name(model)

In [7]:
from utils import SubScafLinear, SubScafSGD
import math
import torch.nn as nn
import torch
def gene_random_matrix(in_dim, out_dim):
    return torch.randn(in_dim, out_dim) / math.sqrt(out_dim)

In [20]:
comp_mat = gene_random_matrix(512, 64).to("cuda:0")
module = nn.Linear(512, 512, device='cuda:0')
model = SubScafLinear(64, comp_mat, module)

In [ ]:
param_groups = model.parameters()
optimizer = SubScafSGD(param_groups, 
                        lr=0.001, 
                        tau=16, 
                        compression_dim=64,
                        )
data = torch.randn(1024, 512).to('cuda:0')
label = torch.randn(1024, 512).to('cuda:0')

In [ ]:
model(data)

torch.Size([1024, 512])

In [48]:
def Coordinate_descend_genep(dim, comp_dim, n=1):
    assert dim >= comp_dim, "compression dimension must be smaller than dimension"
    sum_p = torch.zeros(dim, comp_dim) 
    for _ in range(n):
        ide = torch.eye(dim)
        select_col = torch.randperm(dim)[:comp_dim]
        sign = torch.randint(0, 2, (comp_dim, ))
        sign = sign * 2 - 1
        P = ide[:, select_col] * sign
        sum_p += P
    P = sum_p / n
    return P

In [34]:
import numpy as np
torch.randperm(3)
np.random.choice(10, 3)

array([4, 9, 8])

In [78]:
a = Coordinate_descend_genep(10, 5, 1)

In [ ]:
a.T @ a

tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]])